<a href="https://colab.research.google.com/github/chasslayy/Jua-Shade/blob/main/JuaShade_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JuaShade — Baseline Skin Detection (HSV / YCrCb)

This notebook gives you a **fast, Mid‑Check‑In ready** baseline for your Computer Vision project.

### What you'll get
- Simple classical CV baseline (HSV & YCrCb thresholding)
- Preprocessing (optional: gray‑world color constancy)
- Visualizations (input → mask → overlay)
- Metrics (IoU, Precision, Recall, F1) **if you provide ground‑truth masks**
- A small CLI-style batch runner to process folders

### Quick Start
1. Mount / Upload your dataset (images + optional masks)
2. Set the `DATA_DIR` paths in the **Config** cell
3. Run cells top‑to‑bottom and collect outputs/images for your Mid‑Check‑In slides

> Tip: If you don't have masks yet, you can still demo qualitative results and failure cases—totally acceptable for mid‑check.

## 0. Environment Setup
Install and import packages. (OpenCV may already be present in Colab.)

In [ ]:
try:
    import cv2
except Exception as e:
    # In Colab, this will install OpenCV if missing
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'opencv-python-headless'])
    import cv2

import os, glob, math
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Dict

print('OpenCV version:', cv2.__version__)

OpenCV version: 4.12.0


## 1. Config — Set your paths here

In [ ]:
# 👉 Update these to your dataset paths
DATA_DIR = '/content/images'        # folder of *.jpg/*.png
MASK_DIR = '/content/masks'         # optional: GT masks, same filenames as images
OUT_DIR  = '/content/jua_baseline_outputs'
os.makedirs(OUT_DIR, exist_ok=True)

# File pattern
IMG_EXTS = ('*.jpg','*.jpeg','*.png')

def list_images(folder):
    files = []
    for ext in IMG_EXTS:
        files.extend(glob.glob(os.path.join(folder, ext)))
    files.sort()
    return files

image_paths = list_images(DATA_DIR)
print(f'Found {len(image_paths)} images in {DATA_DIR}')

Found 0 images in /content/images


## 2. Utilities — I/O, visualization, metrics

In [ ]:
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def read_mask(path):
    m = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if m is None:
        return None
    # Binarize just in case
    _, m = cv2.threshold(m, 127, 255, cv2.THRESH_BINARY)
    return (m > 127).astype(np.uint8)

def show_triplet(img, mask, overlay_title='Overlay'):
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(img); plt.title('Input'); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(mask, cmap='gray'); plt.title('Predicted Mask'); plt.axis('off')
    # overlay in RGB
    overlay = img.copy()
    red = np.zeros_like(img)
    red[...,0] = 255
    overlay = np.where(mask[...,None].astype(bool), (0.6*overlay + 0.4*red).astype(np.uint8), overlay)
    plt.subplot(1,3,3); plt.imshow(overlay); plt.title(overlay_title); plt.axis('off')
    plt.show()

def compute_metrics(y_true, y_pred) -> Dict[str, float]:
    # y_true, y_pred in {0,1}
    y_true = y_true.astype(bool)
    y_pred = y_pred.astype(bool)
    tp = np.logical_and(y_true, y_pred).sum()
    fp = np.logical_and(~y_true, y_pred).sum()
    fn = np.logical_and(y_true, ~y_pred).sum()
    union = np.logical_or(y_true, y_pred).sum()
    iou = tp / union if union else 0.0
    prec = tp / (tp + fp) if (tp+fp) else 0.0
    rec  = tp / (tp + fn) if (tp+fn) else 0.0
    f1 = 2*prec*rec/(prec+rec) if (prec+rec) else 0.0
    return {'IoU': iou, 'Precision': prec, 'Recall': rec, 'F1': f1}

def save_mask(mask01: np.ndarray, out_path: str):
    mask255 = (mask01.astype(np.uint8))*255
    cv2.imwrite(out_path, mask255)

## 3. Optional: Gray-World Color Constancy
This can help reduce lighting bias before thresholding.

In [ ]:
def gray_world(img_rgb: np.ndarray) -> np.ndarray:
    # img_rgb in [0..255]
    img = img_rgb.astype(np.float32)
    avg = img.mean(axis=(0,1)) + 1e-6
    gray = avg.mean()
    scale = gray / avg
    balanced = img * scale
    balanced = np.clip(balanced, 0, 255).astype(np.uint8)
    return balanced

## 4. Baseline A — HSV Thresholding
You can tune the ranges; the defaults cover common skin tones but will not be perfect. That's the point for baseline.

In [ ]:
# Default HSV ranges for skin (broad). Tune per dataset.
# Hue range for skin often ~ [0, 25] U [160, 180] in OpenCV's 0-179 hue scale, but saturation/value gates matter a lot.
HSV_LOWER_1 = np.array([0,   30,  50])
HSV_UPPER_1 = np.array([25, 200, 255])
HSV_LOWER_2 = np.array([160, 30,  50])
HSV_UPPER_2 = np.array([179, 200, 255])

def skin_mask_hsv(img_rgb: np.ndarray, use_grayworld: bool=True) -> np.ndarray:
    x = gray_world(img_rgb) if use_grayworld else img_rgb
    hsv = cv2.cvtColor(x, cv2.COLOR_RGB2HSV)
    m1 = cv2.inRange(hsv, HSV_LOWER_1, HSV_UPPER_1)
    m2 = cv2.inRange(hsv, HSV_LOWER_2, HSV_UPPER_2)
    m = cv2.bitwise_or(m1, m2)
    # Morph cleanup
    kernel = np.ones((5,5), np.uint8)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, kernel, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=1)
    return (m>0).astype(np.uint8)

## 5. Baseline B — YCrCb Thresholding

In [ ]:
# Broad YCrCb thresholds for skin detection; tune per dataset
YCRCB_LOWER = np.array([0,   135,  85])
YCRCB_UPPER = np.array([255, 180, 135])

def skin_mask_ycrcb(img_rgb: np.ndarray, use_grayworld: bool=True) -> np.ndarray:
    x = gray_world(img_rgb) if use_grayworld else img_rgb
    ycc = cv2.cvtColor(x, cv2.COLOR_RGB2YCrCb)
    m = cv2.inRange(ycc, YCRCB_LOWER, YCRCB_UPPER)
    kernel = np.ones((5,5), np.uint8)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, kernel, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=1)
    return (m>0).astype(np.uint8)

## 6. Demo on a few images

In [ ]:
SAMPLE_N = 3
for path in image_paths[:SAMPLE_N]:
    img = read_image(path)
    m_hsv = skin_mask_hsv(img)
    m_ycc = skin_mask_ycrcb(img)
    print('→', os.path.basename(path))
    show_triplet(img, m_hsv, overlay_title='HSV Overlay')
    show_triplet(img, m_ycc, overlay_title='YCrCb Overlay')

## 7. Metrics (optional, if you have ground-truth masks)

In [ ]:
def mask_path_from_image(image_path: str) -> str:
    base = os.path.splitext(os.path.basename(image_path))[0]
    for ext in ('.png', '.jpg', '.jpeg'):
        p = os.path.join(MASK_DIR, base + ext)
        if os.path.exists(p):
            return p
    return None

def evaluate_folder(method='hsv', max_items=50):
    metrics = []
    count = 0
    for img_p in image_paths:
        m_p = mask_path_from_image(img_p)
        if not m_p:
            continue
        img = read_image(img_p)
        gt = read_mask(m_p)
        if gt is None:
            continue
        if method=='hsv':
            pred = skin_mask_hsv(img)
        else:
            pred = skin_mask_ycrcb(img)
        metrics.append(compute_metrics(gt, pred))
        count += 1
        if count >= max_items:
            break
    if not metrics:
        print('No masks found; skipping metrics.')
        return None
    # Aggregate
    keys = metrics[0].keys()
    avg = {k: float(np.mean([m[k] for m in metrics])) for k in keys}
    print('Average metrics (', method, '):', avg)
    return avg

# Example (will only print if masks exist):
evaluate_folder('hsv')
evaluate_folder('ycrcb')

No masks found; skipping metrics.
No masks found; skipping metrics.


## 8. Batch Runner — Save masks to a folder

In [ ]:
def process_folder(method='hsv', limit=None):
    os.makedirs(OUT_DIR, exist_ok=True)
    n = 0
    for img_p in image_paths:
        img = read_image(img_p)
        if method=='hsv':
            pred = skin_mask_hsv(img)
        else:
            pred = skin_mask_ycrcb(img)
        out_p = os.path.join(OUT_DIR, os.path.splitext(os.path.basename(img_p))[0] + f'_{method}_mask.png')
        save_mask(pred, out_p)
        n += 1
        if limit and n>=limit:
            break
    print(f'Saved {n} masks to {OUT_DIR}')

# Example run (commented):
# process_folder('hsv', limit=20)

## 9. Next Steps (for Mid‑Check‑In slide)
- Show **2–3** side‑by‑side results where HSV and YCrCb succeed/fail differently.
- Report IoU/F1 if masks are available; otherwise share qualitative overlays.
- Note challenges: lighting, deeper skin tones missed by default thresholds, backgrounds.
- Plan your improved model (U‑Net, lightweight CNN, or segmentation model).